# सत्र 1 – चैट बूटस्ट्रैप (फाउंड्री लोकल)

यह नोटबुक फाउंड्री लोकल को बूटस्ट्रैप करता है, पसंदीदा मॉडल उपनाम डाउनलोड करता है, और एक मानक और स्ट्रीमिंग चैट पूर्णता दोनों को निष्पादित करता है।


# परिदृश्य
यह सत्र आपको Foundry Local के माध्यम से एक स्थानीय छोटे भाषा मॉडल को प्रतिक्रिया देने के लिए आवश्यक न्यूनतम जानकारी प्रदान करता है। आप निम्नलिखित करेंगे:
- SDK / क्लाइंट डिपेंडेंसीज़ इंस्टॉल करें।
- चुने गए उपनाम (डिफ़ॉल्ट: `phi-4-mini`) के लिए Foundry Local मैनेजर को इनिशियलाइज़ करें।
- मॉडल मेटाडेटा में वैकल्पिक फ़ील्ड को सहन करने के लिए एक डिफेंसिव मंकी-पैच लागू करें।
- एक मानक चैट पूर्णता अनुरोध भेजें।
- प्रतिक्रिया को टोकन‑दर‑टोकन स्ट्रीम करें।

उद्देश्य यह है कि RAG, रूटिंग, या एजेंट्स पर जाने से पहले आपके स्थानीय रनटाइम और नेटवर्क पथ को मान्य किया जाए।


### व्याख्या: निर्भरता स्थापना
इस न्यूनतम चैट फ्लो के लिए आवश्यक Python पैकेज स्थापित करता है:
- `foundry-local-sdk`: स्थानीय मॉडल और सेवा जीवनचक्र प्रबंधन।
- `openai`: चैट पूर्णता के लिए परिचित क्लाइंट अमूर्त।
- `rich`: स्पष्ट नोटबुक आउटपुट के लिए सुंदर प्रिंटिंग।

पुनः चलाना सुरक्षित है (idempotent)। यदि आपके वातावरण में पहले से ही ये हैं, तो इसे छोड़ दें।


In [1]:
# Install required libraries (idempotent)
%pip install -q foundry-local-sdk openai rich


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### व्याख्या: मुख्य आयात
नोटबुक में उपयोग किए जाने वाले मॉड्यूल लाता है:
- `FoundryLocalManager` स्थानीय मॉडल रनटाइम के साथ इंटरफेस करने के लिए।
- `OpenAI` क्लाइंट ताकि हम परिचित चैट पूर्णता API सतह का पुन: उपयोग कर सकें।
- `rich.print` स्टाइलिश आउटपुट के लिए।

यहां कोई नेटवर्क कॉल नहीं होती—यह केवल नेमस्पेस तैयार करता है।


In [2]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

### व्याख्या: मैनेजर इनिशियलाइज़ेशन और मेटाडेटा पैच
चुने गए उपनाम के लिए `FoundryLocalManager` को इनिशियलाइज़ करता है और एक सुरक्षात्मक मंकी-पैच लागू करता है ताकि उन सेवा प्रतिक्रियाओं को सहजता से संभाला जा सके जहां `promptTemplate` संभवतः `null` हो सकता है।

मुख्य परिणाम:
- सेवा की स्थिति और एंडपॉइंट की पुष्टि करता है।
- कैश किए गए मॉडल्स की सूची दिखाता है (स्थानीय स्टोर को सत्यापित करता है)।
- उपनाम के लिए ठोस मॉडल ID को हल करता है (जो बाद के चैट कॉल्स में उपयोग होता है)।

यदि आपको कच्चे सेवा मेटाडेटा में सत्यापन समस्याओं का सामना करना पड़ता है, तो यह पैटर्न दिखाता है कि SDK को फोर्क किए बिना इसे कैसे साफ किया जाए।


In [3]:
# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:50262/v1

Cached models:
[
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-generic-gpu:4',
        version='4',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-generic-gpu/versions/4',
        file_size_mb=3809,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-generic-gpu:3',
        version='3',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-generic-gpu/versions/3',
        file_size_mb=700,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-generic-gpu:1',
        version='1',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-generic-gpu/versions/1',
        file_size_mb=2211,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    )
]

Using model id: Phi-4-mini-instruct-generic-gpu:4

### व्याख्या: बेसिक चैट पूर्णता
स्थानीय Foundry एंडपॉइंट की ओर इशारा करते हुए एक `OpenAI`-संगत क्लाइंट बनाता है और एक गैर-स्ट्रीमिंग चैट पूर्णता करता है। यहाँ ध्यान दें:
- सुनिश्चित करें कि मॉडल बिना किसी त्रुटि के प्रतिक्रिया देता है।
- विलंबता / आउटपुट प्रारूप को मान्य करें।
- संसाधनों को बचाने के लिए `max_tokens` को सीमित रखें।

यदि यह विफल होता है, तो पुनः जांचें कि Foundry Local सेवा चल रही है और उपनाम सही ढंग से हल हो रहा है।


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':prompt}],
    max_tokens=120,
    temperature=0.5
)
print(resp.choices[0].message.content)

Local inference for privacy refers to the practice of performing data analysis on a local device without sending 
sensitive information to a central server. Two benefits of this approach are:


1. **Enhanced Privacy**: Local inference keeps personal data on the user's device, reducing the risk of data 
breaches and unauthorized access. Since the data is not transmitted over the network, it is less susceptible to 
interception by malicious actors.


2. **Data Sovereignty**: Users retain control over their data, as it does not leave their device. This means that 
individuals or organizations can comply with local data protection regulations, such as the General

### व्याख्या: स्ट्रीमिंग चैट पूर्णता
बेहतर अनुभवित विलंबता और इंटरैक्टिव UX के लिए टोकन स्ट्रीमिंग का प्रदर्शन करता है। लूप इनक्रिमेंटल डेल्टा को प्रिंट करता है जैसे ही वे आते हैं:
- उन चैट UI के लिए उपयोगी जहां प्रारंभिक आंशिक आउटपुट महत्वपूर्ण होता है।
- आपको टोकन थ्रूपुट बनाम पूर्ण पूर्णता विलंबता को मापने की अनुमति देता है।

आप इस पैटर्न को टोकन जमा करने, प्रगति विजेट को अपडेट करने, या मध्य-पीढ़ी में प्रक्रिया को रोकने के लिए अनुकूलित कर सकते हैं।


In [5]:
# Streaming example
stream = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}],
    stream=True,
    max_tokens=60,
    temperature=0.4
)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

latency

and

bandwidth

usage

.

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
